## SQL

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('sparksql') \
    .getOrCreate()

In [24]:
# Spark dfs
df_green = spark.read.parquet('data/pq/green/*/*')
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [36]:
print("Green Schema")
df_green.printSchema()
print("Yellow Schema")
df_yellow.printSchema()

Green Schema
root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

Yellow Schema
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_p

In [47]:
# Rename Columns so both df can benefit from the time stamps in common columns
df_green = df_green.withColumnsRenamed(
    {
        "lpep_pickup_datetime":"pickup_datetime",
        "lpep_dropoff_datetime":"dropoff_datetime",
    }
)

df_yellow = df_yellow.withColumnsRenamed(
    {
        "tpep_pickup_datetime":"pickup_datetime",
        "tpep_dropoff_datetime":"dropoff_datetime",
    }
)

In [48]:
# use set to get the intersection of both green and yellow columns
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [53]:
# For preserving the correct columns order.
common_columns = []
for column in df_green.columns:
    if column in df_yellow.columns:
        common_columns.append(column)
common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [ ]:
# Storing the columns for further usage.
common_columns = ['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [62]:
# Prepare the datasets with preserved column order and add a service type column
df_green_sort = df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('Green'))

df_yellow_sort = df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('Yellow'))



In [66]:
# Union both datasets
df_trips_data = df_green_sort.unionAll(df_yellow_sort)

In [72]:
# Testing
df_trips_data \
    .groupBy('service_type') \
    .count().show()

[Stage 13:====================================================>   (14 + 1) / 15]

+------------+--------+
|service_type|   count|
+------------+--------+
|       Green| 2304517|
|      Yellow|39649199|
+------------+--------+



In [ ]:
# This is deprecated 
# df_trips_data.registerTempTable('trips_data')

In [80]:
# Use this instead
# df needs a temporary view so it can be used in the sql block bellow
df_trips_data.createOrReplaceTempView('trips_data')

In [82]:
spark.sql(
    """
    SELECT 
        service_type, 
        count(*) 
    FROM 
        trips_data 
    GROUP BY service_type;
    """
).show()

[Stage 25:================================================>       (13 + 1) / 15]

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       Green| 2304517|
|      Yellow|39649199|
+------------+--------+



In [84]:
df_result = spark.sql(
    """
    SELECT 
        -- Reveneue grouping 
        PULocationID AS revenue_zone,
        date_trunc('month', pickup_datetime) AS revenue_month, 
        service_type, 
    
        -- Revenue calculation 
        SUM(fare_amount) AS revenue_monthly_fare,
        SUM(extra) AS revenue_monthly_extra,
        SUM(mta_tax) AS revenue_monthly_mta_tax,
        SUM(tip_amount) AS revenue_monthly_tip_amount,
        SUM(tolls_amount) AS revenue_monthly_tolls_amount,
        SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
        SUM(total_amount) AS revenue_monthly_total_amount,
        SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,
    
        -- Additional calculations
        AVG(passenger_count) AS avg_montly_passenger_count,
        AVG(trip_distance) AS avg_montly_trip_distance
    FROM
        trips_data
    GROUP BY
        1, 2, 3
    """
)

In [88]:
# wirte the results to a parquet file.
local_path='data/report/revenue'

df_result \
    .coalesce(1) \
    .write.parquet(local_path, mode='overwrite')